In [10]:
import dash
from dash import Dash, dcc, html
from dash.dependencies import Input, Output
from dash.exceptions import PreventUpdate
from jupyter_dash import JupyterDash
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import seaborn as sns
import pandas as pd
import numpy as np
import sys

np.set_printoptions(threshold=sys.maxsize)


In [ ]:
data = pd.read_csv("NY-House-Dataset.csv")
data.head(100)

In [ ]:
data.info()

BROKERTITLE                     object
TYPE                            object
PRICE                            int64
BEDS                             int64
BATH                           float64
PROPERTYSQFT                   float64
ADDRESS                         object
STATE                           object
MAIN_ADDRESS                    object
ADMINISTRATIVE_AREA_LEVEL_2     object
LOCALITY                        object
SUBLOCALITY                     object
STREET_NAME                     object
LONG_NAME                       object
FORMATTED_ADDRESS               object
LATITUDE                       float64
LONGITUDE                      float64
dtype: object

* BROKERTITLE: Title of the broker
* TYPE: Type of the house
* PRICE: Price of the house
* BEDS: Number of bedrooms
* BATH: Number of bathrooms
* PROPERTYSQFT: Square footage of the property
* ADDRESS: Full address of the house
* STATE: State of the house
* MAIN_ADDRESS: Main address information
* ADMINISTRATIVE_AREA_LEVEL_2: Administrative area level 2 information
* LOCALITY: Locality information
* SUBLOCALITY: Sublocality information
* STREET_NAME: Street name
* LONG_NAME: Long name
* FORMATTED_ADDRESS: Formatted address
* LATITUDE: Latitude coordinate of the house
* LONGITUDE: Longitude coordinate of the house

In [ ]:
data["BROKERTITLE"].unique()
data["TYPE"].unique()
data["PRICE"].unique()
data["BEDS"].unique()
data["BATH"].unique()
data["PROPERTYSQFT"].unique()
data["ADDRESS"].unique()
data["STATE"].unique()
data["MAIN_ADDRESS"].unique()
data["ADMINISTRATIVE_AREA_LEVEL_2"].unique()
data["LOCALITY"].unique()
data["SUBLOCALITY"].unique()
data["STREET_NAME"].unique()
data["LONG_NAME"].unique()
# data["FORMATTED_ADDRESS"].unique()
# data["LATITUDE"].unique()
# data["LONGITUDE"].unique()


In [14]:
datax = data.copy()

### column BROKERTITLE:

1. delete all 'Brokered by'
2. delete leading and trailing whitespaces


In [15]:
datax['BROKERTITLE1'] = datax['BROKERTITLE'].str.replace('Brokered by ', "").str.strip()


### column TYPE:

https://www.manhattanmiami.com/resources/home-buying-guide/co-op-vs-condo

### columns BATH, PROPERTYSQFT:

strange value = 2.37386086  ??? rounded!
in csv = 2.3738608579684373


In [ ]:
datax.loc[datax['BATH'] == 2.3738608579684373, :]


In [17]:
datax['BATH1'] = datax['BATH'].astype('int64')

In [18]:
datax['BATH1'].unique()

array([ 2, 10,  1, 16,  3,  4,  6,  8,  5,  9,  7, 32, 13, 50, 20, 11, 12,
       24, 43,  0, 17])

In [ ]:
datax.loc[datax['PROPERTYSQFT'] == 2184.207862, :]

In [ ]:
datax['PROPERTYSQFT1'] = datax['PROPERTYSQFT'].astype('int64')
datax['PROPERTYSQFT1'].unique()

### column ADDRESS, STATE, MAIN_ADDRESS:

In [21]:
datax['ADDRESS1'] = datax['ADDRESS'].str.strip()
datax['STATE1'] = datax['STATE'].str.strip()
datax['MAIN_ADDRESS1'] = datax['MAIN_ADDRESS'].str.strip()

### columns of location hierarchy:


* ZIP - using regex formula to extract zip code from 'FORMATTED_ADDRESS' column

* COUNTY, CITY - using zipcodes module to extract counties/cities from it (using for loop to change each result for each zip code from list of dictionaries to a dictionary and make a column by list.append(dict.get('key')))

* STREET_NAME - using column FORMATTED_ADDRESS to extract names of streets (using package 'usaddress', I have also tried to use geopy.geocoders - Nominatim based on lat and long, but it is inapplicable due to amount of rows)

* delete ['ADMINISTRATIVE_AREA_LEVEL_2', 'LOCALITY', 'SUBLOCALITY'] = inconsistent, and replace them by new columns 

United States is structured as follows: Admin 0 - Nation, Admin 1 States along with Federal District (DC), Admin 2 - Counties & Independent Cities, etc.

In [22]:
! pip install zipcodes


In [23]:
import zipcodes
print(zipcodes.matching('11214'))

[{'zip_code': '11214', 'zip_code_type': 'STANDARD', 'active': True, 'city': 'Brooklyn', 'acceptable_cities': [], 'unacceptable_cities': [], 'state': 'NY', 'county': 'Kings County', 'timezone': 'America/New_York', 'area_codes': ['718'], 'world_region': 'NA', 'country': 'US', 'lat': '40.5977', 'long': '-74.0010'}]


In [24]:
omg = zipcodes.matching('11214')


In [25]:
type(omg) # omg is a list of dictionaries

list

In [26]:
import re
datax['ZIP'] = datax['FORMATTED_ADDRESS'].str.extract('(?<=NY )(.*?)(?=, USA)', expand=True)


In [29]:
county = []

for value in datax["ZIP"]:
    dict = {}
    for i in zipcodes.matching(value):
        dict.update(i)
    county.append(dict.get('county'))

datax['COUNTY'] = county

In [32]:
city = []

for value in datax["ZIP"]:
    dict1 = {}
    for i in zipcodes.matching(value):
        dict1.update(i)
    city.append(dict1.get('city'))

datax['CITY'] = city

In [37]:
datax.drop(['ADMINISTRATIVE_AREA_LEVEL_2', 'LOCALITY', 'SUBLOCALITY'], inplace=True, axis=1)
# datax.drop(['BROKERTITLE', 'BATH', 'PROPERTYSQFT', 'ADDRESS', 'STATE', 'MAIN_ADDRESS'], inplace=True, axis=1)

In [40]:
# ! pip install geopy

In [46]:
# from geopy.geocoders import Nominatim
# from pprint import pprint

# # Instantiate a new Nominatim client
# geolocator = Nominatim(user_agent="NY_address")

# # Get location raw data from the user

# location = geolocator.reverse("40.761255, -73.974483")
# print(location.address)

Astor Court, 2, East 55th Street, Manhattan Community Board 5, Manhattan, New York County, City of New York, New York, 10022, United States


In [47]:
# datax['COORDINATES'] = datax['LATITUDE'].astype(str) + ', ' + datax['LONGITUDE'].astype(str)

In [125]:
# addressLL = []

# for value in datax['COORDINATES']:
#     geolocator = Nominatim(user_agent="NY_address")
#     location = geolocator.reverse(value)
#     addressLL.append(location.address)

# datax.drop('COORDINATES', inplace=True, axis=1)

In [ ]:
! pip install usaddress

In [66]:
import usaddress

In [98]:
# addr='217 W 57th St, New York, NY 10019, USA'
# test = usaddress.parse(addr)
# print(test)
# # test2 = dict(test)
# # print(test2)
# result1 = [item for item in test if item[1] == 'StreetName']
# result2 = [item for item in test if item[1] == 'StreetNamePostType']
# print(result1[0][0]  + ' ' + result2[0][0])

[('217', 'AddressNumber'), ('W', 'StreetNamePreDirectional'), ('57th', 'StreetName'), ('St,', 'StreetNamePostType'), ('New', 'PlaceName'), ('York,', 'PlaceName'), ('NY', 'StateName'), ('10019,', 'ZipCode'), ('USA', 'CountryName')]
57th St,


In [338]:
# street_name = []

# for value in datax['ADDRESS1']:
#     try:
#         res1 = [item for item in usaddress.parse(value) if item[1] == 'StreetName']
#         res2 = [item for item in usaddress.parse(value) if item[1] == 'StreetNamePostType']
#         street_name.append(res1[0][0] + ' ' + res2[0][0])

#     except IndexError as e:
#             street_name.append('NaN')

# datax['STR_NAME'] = street_name

# datax.drop('STREET_NAME', inplace=True, axis=1)
# datax.drop('LONG_NAME', inplace=True, axis=1)

In [194]:
# datax['STR_NAME2'] = datax['STR_NAME'].str.replace(r"\W", " ", regex=True).str.strip()

In [459]:
test = []
for value in datax['FORMATTED_ADDRESS']:
    try:
        res1 = [item for item in usaddress.parse(value) if item[1] == 'StreetName']
        res2 = [item for item in usaddress.parse(value) if item[1] == 'StreetNamePostType']
        test.append(res1[0][0] + ' ' + res2[0][0])
    except IndexError:
        if 'Avenue'in value:
            test.append(re.findall(r"(Avenue\s.)", value)[0])
        elif 'Broadway' in value:
            test.append('Broadway')
        elif '55th St' in value:
            test.append('55th St') 
        elif 'Central Park' in value:
            test.append(re.findall(r"(Central Park\s.)", value)[0])
        elif 'Prospect Park W' in value:
            test.append('Prospect Park W')
        elif 'Tudor City' in value:
            test.append('Tudor City Pl')
        elif 'South St' in value:
            test.append('South St')
        elif '164th Dr' in value:
            test.append('164th Dr')
        elif 'Ocean Promenade' in value:
            test.append('Ocean Promenade')
        elif 'Fifth Ave' in value:
            test.append('Fifth Ave')
        elif 'Bowery' in value:
            test.append('Bowery')
        elif 'Grand Concourse' in value:
            test.append('Grand Concourse')
        elif 'Morris Park W' in value:            
            test.append('Morris Park W')
        elif 'First Ave' in value:
            test.append('First Ave')
        elif 'Court Sq W' in value:
            test.append('Court Sq W')
        elif 'Northside Piers' in value:
            test.append('Northside Piers')
        elif 'Alden Park' in value:
            test.append('Alden Park')
        elif 'Hudson Yards' in value:
            test.append('Hudson Yards')
        elif 'Ave W' in value:
            test.append('Avenue W')
        elif 'Gramercy Park S' in value:
            test.append('Gramercy Park S')
        elif 'Ave V' in value:
            test.append('Avenue V')
        elif 'Ave Y' in value:
            test.append('Avenue Y')
        elif 'Ave. S' in value:
            test.append('Avenue S')
        elif '5th Ave' in value:
            test.append('5th Avenue')
        elif 'Ocean Prom' in value:
            test.append('Ocean Promenade')
        elif '1st Ave' in value:
            test.append('1st Avenue')
        elif 'Ct Square W' in value:
            test.append('Court Square W')
        elif 'Gramercy Pk S' in value:
            test.append('Gramercy Park S')
        else:
            test.append('NaN')

datax['TEST'] = test



In [ ]:
datax['STR_NAME'] = datax['TEST'].str.replace(r"\W", " ", regex=True).str.strip()

In [463]:
datax.drop('TEST', inplace=True, axis=1)

In [483]:
datax.loc[datax['STR_NAME'] == 'NaN', :]

,TYPE,PRICE,BEDS,FORMATTED_ADDRESS,LATITUDE,LONGITUDE,BROKERTITLE,BATH,PROPERTYSQFT,ADDRESS,STATE,MAIN_ADDRESS,ZIP,COUNTY,CITY,STR_NAME


### Change column names and sort them properly

In [481]:
# datax.columns.tolist()

# datax.rename(columns = {'BROKERTITLE1': 'BROKERTITLE',
#                         'BATH1': 'BATH',
#                         'PROPERTYSQFT1': 'PROPERTYSQFT',
#                         'ADDRESS1': 'ADDRESS',
#                         'STATE1': 'STATE',
#                         'MAIN_ADDRESS1': 'MAIN_ADDRESS'}, 
#                         inplace = True)

new_cols = ['BROKERTITLE', 
            'TYPE',
            'PRICE',
            'BEDS',
            'BATH',
            'PROPERTYSQFT',
            'ADDRESS',
            'MAIN_ADDRESS',
            'FORMATTED_ADDRESS',
            'STATE',
            'ZIP',
            'COUNTY',
            'CITY',
            'STR_NAME',
            'LATITUDE',
            'LONGITUDE']

data_cl = datax[new_cols]


In [482]:
data_cl.head()

,BROKERTITLE,TYPE,PRICE,BEDS,BATH,PROPERTYSQFT,ADDRESS,MAIN_ADDRESS,FORMATTED_ADDRESS,STATE,ZIP,COUNTY,CITY,STR_NAME,LATITUDE,LONGITUDE
0,Douglas Elliman -111 Fifth Ave,Condo for sale,315000,2,2,1400,2 E 55th St Unit 803,"2 E 55th St Unit 803New York, NY 10022","Regis Residence, 2 E 55th St #803, New York, N...","New York, NY 10022",10022,New York County,New York,55th St,40.761255,-73.974483
1,Serhant,Condo for sale,195000000,7,10,17545,Central Park Tower Penthouse-217 W 57th New Yo...,Central Park Tower Penthouse-217 W 57th New Yo...,"217 W 57th St, New York, NY 10019, USA","New York, NY 10019",10019,New York County,New York,57th St,40.766393,-73.980991
2,Sowae Corp,House for sale,260000,4,2,2015,620 Sinclair Ave,"620 Sinclair AveStaten Island, NY 10312","620 Sinclair Ave, Staten Island, NY 10312, USA","Staten Island, NY 10312",10312,Richmond County,Staten Island,Sinclair Ave,40.541805,-74.196109
3,COMPASS,Condo for sale,69000,3,1,445,2 E 55th St Unit 908W33,"2 E 55th St Unit 908W33Manhattan, NY 10022","2 E 55th St, New York, NY 10022, USA","Manhattan, NY 10022",10022,New York County,New York,55th St,40.761398,-73.974613
4,Sotheby's International Realty - East Side Man...,Townhouse for sale,55000000,7,2,14175,5 E 64th St,"5 E 64th StNew York, NY 10065","5 E 64th St, New York, NY 10065, USA","New York, NY 10065",10065,New York County,New York,64th St,40.767224,-73.969856
